# Creación de un mapa de contaminación

##### Se pretende cargar la posición de las estaciones de medición de contaminantes junto con sus contaminantes y realizar una interpolación de estos contaminantes en todo un mallado entre las estaciones conocidas.

In [1]:
import numpy as np
import pandas as pd
import re
from scipy.interpolate import SmoothBivariateSpline
import matplotlib.pyplot as plt 

In [2]:
def dms2dd(input):
    "Esta función tranforma un punto expresado en coordenadas geográficas a coordenadas UTM"
    degrees,minutes,seconds,direction=re.split('[º\´"]+',str(input))
    dd = float(degrees) + float(minutes)/60 + float(seconds)/(60*60);
    if direction == 'S' or direction == 'W':
        dd *= -1
    return dd;

##### Se leeran las coordenadas de las estaciones y se filtrarán las que son de nuestro interés

In [3]:
posicion_estaciones=pd.read_excel('./data/EstacionesLocalizacion.xlsx') #Se carga el fichero

In [4]:
Estacion=[4,8,11,35,38,39,48,49,50] #Estaciones para nuestro estudio

In [5]:
posicion_estaciones=posicion_estaciones[posicion_estaciones['Estacion'].isin(Estacion)]

In [6]:
posicion_estaciones.set_index('Estacion',inplace=True)

##### Se convertirán los valores de Latitud y Longitud en coordenadas decimales

In [7]:
posicion_estaciones[['Latitud','Longitud']]=posicion_estaciones[['Latitud','Longitud']].applymap(lambda x: dms2dd(x))

In [8]:
posicion_estaciones.sort_values(['Latitud','Longitud'])

,Columna1,Latitud,Longitud
Estacion,,,
49,Retiro,40.414444,3.682583
35,Plaza del carmen,40.419208,3.703167
8,Escuelas Aguirre,40.421564,3.682283
4,Plaza de España,40.423883,3.712197
48,Paseo Castellana,40.439892,3.690372
38,Cuatro Caminos,40.445547,3.707122
11,Av Ramon y Cajal,40.451469,3.677361
50,Plaza Castilla,40.465583,3.688744
39,Barrio del pilar,40.478233,3.711536


In [9]:
sortLatitud=posicion_estaciones['Latitud'].sort_values()
print(sortLatitud)
sortLongitud=posicion_estaciones['Longitud'].sort_values()
print(sortLongitud)

Estacion
49    40.414444
35    40.419208
8     40.421564
4     40.423883
48    40.439892
38    40.445547
11    40.451469
50    40.465583
39    40.478233
Name: Latitud, dtype: float64
Estacion
11    3.677361
8     3.682283
49    3.682583
50    3.688744
48    3.690372
35    3.703167
38    3.707122
39    3.711536
4     3.712197
Name: Longitud, dtype: float64


##### Se desea obtener una matriz de puntos sobre la que realizar una interpolación. La interpolación se realizará sobre los puntos conocidos, para obtener una matriz de valores contaminantes, que representará la polución en cada espacio geografico de Madrid capital

In [10]:
Latitudes=[]
Longitudes=[]
for i in range(0,(len(sortLatitud))):
    if i==8:
        lastLat=np.round(sortLatitud.iloc[i],decimals=6)
        lastLon=np.round(sortLongitud.iloc[i],decimals=6)
        Latitudes.append(lastLat)
        Longitudes.append(lastLon)
    else:
        inicioLat=sortLatitud.iloc[i]
        inicioLon=sortLongitud.iloc[i]
        finLat=sortLatitud.iloc[i+1]
        finLon=sortLongitud.iloc[i+1]
        betweenLat=np.round(np.arange(inicioLat,finLat,0.001),decimals=6)
        betweenLon=np.round(np.arange(inicioLon,finLon,0.001),decimals=6)
        Latitudes.extend(betweenLat)
        Longitudes.extend(betweenLon)

In [11]:
datos_estaciones=pd.read_csv('./data/NO2_estaciones.csv')
datos_estaciones.head()

,FECHA,Estacion4,Estacion8,Estacion11,Estacion35,Estacion38,Estacion39,Estacion48,Estacion49,Estacion50
0,2015-01-01 01:00:00,83.0,168.0,93.0,109.0,102.0,104.0,108.0,136.0,127.0
1,2015-01-01 02:00:00,77.0,203.0,224.0,133.0,183.0,168.0,114.0,111.0,170.0
2,2015-01-01 03:00:00,86.0,220.0,152.0,126.0,193.0,203.0,141.0,109.0,118.0
3,2015-01-01 04:00:00,65.0,169.0,151.0,112.0,122.0,147.0,145.0,99.0,126.0
4,2015-01-01 05:00:00,53.0,154.0,106.0,109.0,97.0,159.0,142.0,103.0,140.0


In [12]:
fecha=list(datos_estaciones.FECHA)

In [13]:
ix1 = pd.MultiIndex.from_product([fecha,Latitudes])

In [14]:
mapa = pd.DataFrame(index=ix1, columns=Longitudes, dtype=np.float64)

In [21]:
print(len(Latitudes),len(Longitudes),mapa.loc[(date),:].shape)

69 39 (69, 39)


In [24]:
#for date in fechas 
date='2015-01-01 01:00:00'
x=[]
y=[]
Z=[]
for i in Estacion:
    lat=posicion_estaciones.loc[i,'Latitud'].round(decimals=6)
    print(lat)
    lon=posicion_estaciones.loc[i,'Longitud'].round(decimals=6)
    print(lon)
    Est='Estacion'+str(i)
    sust=float(datos_estaciones[datos_estaciones['FECHA']==date][Est])
    #mapa.loc[(date,lat),lon]=sust
    x.append(lon)
    y.append(lat)
    Z.append(sust)
    
interp_spline = SmoothBivariateSpline(y,x,Z,kx=2,ky=2)
mapa.loc[(date),:] = interp_spline(Latitudes,Longitudes)


40.423883
3.712197
40.421564
3.682283
40.451469
3.677361
40.419208
3.703167
40.445547
3.707122
40.478233
3.711536
40.439892
3.690372
40.414444
3.682583
40.465583
3.688744


In [25]:
mapa.loc[(date),:]

,3.677361,3.678361,3.679361,3.680361,3.681361,3.682283,3.682583,3.683583,3.684583,3.685583,...,3.704167,3.705167,3.706167,3.707122,3.708122,3.709122,3.710122,3.711122,3.711536,3.712197
40.414444,95.810652,104.780927,113.147642,120.910797,128.070393,134.136760,136.000000,141.818486,147.033411,151.644777,...,127.509885,120.301135,112.488826,104.464639,95.472370,85.876541,75.677153,64.874204,60.225123,52.587876
40.415444,111.344163,118.144031,124.448516,130.257620,135.571341,140.031664,141.392158,145.605142,149.322743,152.544963,...,122.279813,115.800474,108.825754,101.702451,93.759259,85.320685,76.386729,66.957392,62.908649,56.268348
40.416444,126.011086,130.751315,135.099078,139.054376,142.617209,145.554401,146.438164,149.128939,151.427249,153.333093,...,117.332630,111.552435,105.379775,99.118513,92.178583,84.846188,77.121328,69.004003,65.528556,59.840156
40.417444,139.811421,142.602780,145.099329,147.301067,149.207996,150.704970,151.138017,152.389878,153.346928,154.009169,...,112.668336,107.557017,102.150889,96.712827,90.730344,84.453052,77.880949,71.014037,68.084845,63.303300
40.418444,152.745167,153.698425,154.449267,154.997693,155.343702,155.483373,155.491717,155.387957,155.081782,154.573190,...,108.286932,103.814222,99.139095,94.485391,89.414541,84.141275,78.665593,72.987494,70.577514,66.657781
40.419208,162.042601,161.666191,161.154409,160.507254,159.724726,158.883289,158.584689,157.501364,156.282665,154.928594,...,105.130164,101.124954,96.984373,92.903745,88.498510,83.957902,79.281921,74.470568,72.439044,69.147380
40.420208,173.447685,171.428571,169.357197,167.233563,165.057669,163.005190,162.327776,160.035761,157.691486,155.294951,...,101.247776,97.827784,94.355531,90.990745,87.416324,83.789644,80.110704,76.379504,74.819491,72.310178
40.421208,183.986181,180.435131,176.909673,173.409806,169.935531,166.754924,165.724710,162.307299,158.915480,155.549252,...,97.648278,94.783235,91.943783,89.255996,86.466575,83.702746,80.964509,78.251863,77.136318,75.364311
40.421564,187.528719,183.459036,179.441390,175.475780,171.562207,168.000000,166.850469,163.052521,159.306611,155.612737,...,96.435137,93.760350,91.137599,88.681449,86.160431,83.691448,81.274503,78.909594,77.945753,76.425354
40.422564,196.892121,191.440705,186.112043,180.906134,175.822979,171.245076,169.778020,164.967624,160.279981,155.715092,...,93.219237,91.058356,89.020229,87.188409,85.390265,83.714875,82.162238,80.732355,80.176313,79.332139
